In [129]:
from source import seasonalregressor
from source import data_wrangle
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import seaborn as sb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
advanced = data_wrangle.read_all_advanced('./data/advanced')
pergame  = data_wrangle.read_all_pergame('./data/per_game')
seasonal = data_wrangle.clean_and_join_seasonal_dataframe(advanced, pergame)
demographic = data_wrangle.read_demographic_data('./data/player_data.csv')
fullstats = data_wrangle.add_years_in_league(seasonal, demographic)

In [3]:
fullstats.head()

,Player,Pos,Age,Tm,G,MP_total,PER,TS%,3PAr,FTr,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Seasons_number
0,A.C. Green,PF,33,TOT,83,2492,12.4,0.523,0.041,0.407,...,2.7,5.2,7.9,0.8,0.8,0.2,0.9,1.7,7.2,12
1,A.C. Green,PF,34,DAL,82,2649,12.8,0.496,0.007,0.303,...,2.7,5.5,8.1,1.5,1.0,0.3,0.8,1.9,7.3,13
2,A.C. Green,PF,35,DAL,50,924,12.5,0.441,0.031,0.203,...,1.6,2.9,4.6,0.5,0.6,0.2,0.4,1.4,4.9,14
3,A.C. Green,PF,36,LAL,82,1929,11.2,0.482,0.010,0.245,...,2.0,4.0,5.9,1.0,0.6,0.2,0.6,1.5,5.0,15
4,A.C. Green,PF,37,MIA,82,1411,11.2,0.492,0.019,0.343,...,1.3,2.5,3.8,0.5,0.4,0.1,0.5,1.5,4.5,16


In [40]:
player = "A.C. Green"
player = "LeBron James"


seasons_needed = {1, 2, 3, 4}
seasons_one_thru_four = {1, 2, 3, 4}
season_to_predict = 5
seasons_needed.add(season_to_predict)

#demographic[demographic['name']==player]['year_start'] == 2015).bool()

count = 0
fullfive = []
notfive = {}
fourbutnotfive_but_not_from2015 = {}
missing_something_from_years_1thru4_but_has_5 ={}
for player in fullstats['Player'].unique():
    playerset = set(fullstats.loc[fullstats['Player']==player, 'Seasons_number'])
    if seasons_needed.issubset(playerset):
        print("Have five full-year stats ",player)
        fullfive.append(player)
        count += 1
    elif seasons_one_thru_four.issubset(playerset) and not (demographic[demographic['name']==player]['year_start'] == 2015).bool():
        fourbutnotfive_but_not_from2015[player] = playerset
    elif 5 in playerset and (demographic[demographic['name']==player]['year_start'] > 1997).bool():
        missing_something_from_years_1thru4_but_has_5[player] = playerset
    else:
        notfive[player] = playerset

print("Number:",count)



Have five full-year stats  A.J. Price
Have five full-year stats  Aaron Gray
Have five full-year stats  Adonal Foyle
Have five full-year stats  Adrian Griffin
Have five full-year stats  Al Harrington
Have five full-year stats  Al Horford
Have five full-year stats  Al Jefferson
Have five full-year stats  Al-Farouq Aminu
Have five full-year stats  Alec Burks
Have five full-year stats  Alex Len
Have five full-year stats  Allen Crabbe
Have five full-year stats  Allen Iverson
Have five full-year stats  Alonzo Gee
Have five full-year stats  Alvin Williams
Have five full-year stats  Amar'e Stoudemire
Have five full-year stats  Amir Johnson
Have five full-year stats  Anderson Varejao
Have five full-year stats  Andray Blatche
Have five full-year stats  Andre Drummond
Have five full-year stats  Andre Iguodala
Have five full-year stats  Andre Miller
Have five full-year stats  Andre Roberson
Have five full-year stats  Andrea Bargnani
Have five full-year stats  Andrei Kirilenko
Have five full-year s

Have five full-year stats  Hasheem Thabeet
Have five full-year stats  Hedo Turkoglu
Have five full-year stats  Hilton Armstrong
Have five full-year stats  Ian Clark
Have five full-year stats  Iman Shumpert
Have five full-year stats  Ira Newble
Have five full-year stats  Isaiah Canaan
Have five full-year stats  Isaiah Thomas
Have five full-year stats  Ish Smith
Have five full-year stats  J.J. Barea
Have five full-year stats  J.J. Hickson
Have five full-year stats  J.J. Redick
Have five full-year stats  J.R. Smith
Have five full-year stats  JaVale McGee
Have five full-year stats  Jacque Vaughn
Have five full-year stats  Jae Crowder
Have five full-year stats  Jahidi White
Have five full-year stats  Jake Tsakalidis
Have five full-year stats  Jake Voskuhl
Have five full-year stats  Jamaal Magloire
Have five full-year stats  Jamaal Tinsley
Have five full-year stats  Jamal Crawford
Have five full-year stats  Jamal Sampson
Have five full-year stats  Jamario Moon
Have five full-year stats  Jame

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

So these are the guys that either need special attention becuase of injuries in years 1-5, or they could be 'zeroed' out in year 5.  Presumably, the GM will know that they aren't giving these guys a 5th year deal.  Like Acie Law, or Anthony Bennett.

In [41]:
for player, years in fourbutnotfive_but_not_from2015.items():
    print("Player:", player, years)

Player: Aaron Brooks {1, 2, 3, 4, 6, 7, 8, 9, 10, 11}
Player: Acie Law {1, 2, 3, 4}
Player: Al Thornton {1, 2, 3, 4}
Player: Andre Barrett {1, 2, 3, 4}
Player: Ansu Sesay {1, 2, 3, 4}
Player: Anthony Bennett {1, 2, 3, 4}
Player: Archie Goodwin {1, 2, 3, 4}
Player: Ben Davis {1, 2, 3, 4}
Player: Carlos Delfino {1, 2, 3, 4, 6, 7, 8, 9}
Player: Chris Copeland {1, 2, 3, 4}
Player: DaJuan Summers {1, 2, 3, 4}
Player: Damion James {1, 2, 3, 4}
Player: Damone Brown {1, 2, 3, 4}
Player: Dan Langhi {1, 2, 3, 4}
Player: Darius Morris {1, 2, 3, 4}
Player: David Harrison {1, 2, 3, 4}
Player: Derrick Rose {1, 2, 3, 4, 6, 7, 8, 9, 10}
Player: Dexter Pittman {1, 2, 3, 4}
Player: Donte Greene {1, 2, 3, 4}
Player: Etdrick Bohannon {1, 2, 3, 4}
Player: Evan Eschmeyer {1, 2, 3, 4}
Player: Felipe Lopez {1, 2, 3, 4}
Player: Gerald Green {1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13}
Player: Greg Stiemsma {1, 2, 3, 4}
Player: Henry Sims {1, 2, 3, 4}
Player: Henry Walker {1, 2, 3, 4, 7}
Player: Hollis Thompson {1, 2,

These are the guys we REALLY need to something special with (smooth, or impute data).  There's actually not that many with data for years one through four BUT not five.

In [42]:
for player, years in fourbutnotfive_but_not_from2015.items():
    if any(x > 5 for x in years):
        print("Player:", player, years)

Player: Aaron Brooks {1, 2, 3, 4, 6, 7, 8, 9, 10, 11}
Player: Carlos Delfino {1, 2, 3, 4, 6, 7, 8, 9}
Player: Derrick Rose {1, 2, 3, 4, 6, 7, 8, 9, 10}
Player: Gerald Green {1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13}
Player: Henry Walker {1, 2, 3, 4, 7}
Player: Ike Diogu {1, 2, 3, 4, 6, 7}
Player: James Anderson {1, 2, 3, 4, 6}
Player: Jeff Green {1, 2, 3, 4, 6, 7, 8, 9, 10, 11}
Player: Jordan Crawford {1, 2, 3, 4, 7, 8}
Player: Josh Childress {1, 2, 3, 4, 7, 8, 9, 10}
Player: Kareem Rush {1, 2, 3, 4, 6, 7, 8}
Player: Kelenna Azubuike {1, 2, 3, 4, 6}
Player: Kerry Kittles {1, 2, 3, 4, 6, 7, 8, 9}
Player: Linas Kleiza {1, 2, 3, 4, 6, 7, 8}


Below are guys that are missing something besides years 5.  Brandon Wright, CDR, Darreell Arthur, Hassan Whiteside, Ian Mahinmi, and similar probably need smoothing. 

In [44]:
for player, years in missing_something_from_years_1thru4_but_has_5.items():
    print(player, years)

Andrew Goudelock {1, 2, 5}
Blake Ahearn {1, 2, 5}
Brandan Wright {1, 2, 4, 5, 6, 7, 8, 9, 10, 11}
Charles Smith (ii) {1, 2, 5, 6, 9}
Charlie Bell {1, 5, 6, 7, 8, 9, 10}
Chris Douglas-Roberts {1, 2, 3, 5, 6, 7}
Corey Benjamin {1, 2, 3, 5}
Dajuan Wagner {1, 2, 3, 5}
Daniel Santiago {1, 2, 4, 5}
Darrell Arthur {1, 2, 3, 5, 6, 7, 8, 9, 10}
Darryl Watkins {1, 5}
DerMarr Johnson {1, 2, 4, 5, 6, 7, 8}
Derrick Dial {1, 2, 3, 5}
Dwight Buycks {1, 2, 5}
Earl Barron {1, 2, 3, 5, 6, 7, 8, 10}
Eddie Gill {1, 2, 4, 5, 6, 8, 9}
Eddie Griffin {1, 2, 4, 5, 6}
Elliot Williams {1, 3, 4, 5}
Ersan Ilyasova {1, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Garrett Temple {1, 2, 4, 5, 6, 7, 8, 9}
Hassan Whiteside {1, 2, 5, 6, 7, 8}
Ian Mahinmi {1, 3, 4, 5, 6, 7, 8, 9, 10, 11}
Ime Udoka {1, 3, 4, 5, 6, 7, 8}
Jabari Smith {1, 2, 4, 5}
James Singleton {1, 2, 4, 5, 7}
Jason Hart {1, 2, 4, 5, 6, 7, 8, 9, 10}
Jason Smith {1, 3, 4, 5, 6, 7, 8, 9, 10, 11}
Jeff Ayres {1, 3, 4, 5, 6, 7}
Jerome James {1, 4, 5, 6, 7, 8, 9, 10, 11}
Jona

In [61]:
SR = seasonalregressor.SeasonalRegressor()

In [63]:
onlyfouryears = fullstats[fullstats['Seasons_number'] == 4]
onlyfiveyears = fullstats[fullstats['Seasons_number'] == 5]
statstokeep = ['WS',
 'VORP',
 'MP_total',
 'PS/G',
 'FG',
 'MP_pergame',
 'GS',
 'FT',
 '2P',
 'FTA',
 'FGA',
 '2PA',
 'DRB', 'Player']


X, y = SR.create_train_and_predict_X_and_y_from_seasons_4_and_5(onlyfouryears, onlyfiveyears, demographic, statstokeep)

In [60]:
import importlib
importlib.reload(seasonalregressor)

<module 'source.seasonalregressor' from '/Users/kv/workspace/kv-capstone/NBAwinshare/source/seasonalregressor.py'>

In [67]:
len(fullfive)

366

In [68]:
fullfive = set(fullfive)

In [81]:
firstfiveyearsfull = fullstats[(fullstats['Player'].isin(fullfive)) &  (fullstats['Seasons_number'] < 6)]

In [86]:
firstfouryearsfull = firstfiveyearsfull[firstfiveyearsfull['Seasons_number'] < 5]

In [88]:
fifthyearfull = firstfiveyearsfull[firstfiveyearsfull['Seasons_number'] == 5]

In [89]:
firstfouryearsfull.head()

,Player,Pos,Age,Tm,G,MP_total,PER,TS%,3PAr,FTr,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Seasons_number
10,A.J. Price,PG,23,IND,56,865,14.0,0.530,0.492,0.212,...,0.2,1.4,1.6,1.9,0.6,0.1,1.1,0.9,7.3,1
11,A.J. Price,PG,24,IND,50,795,10.7,0.454,0.466,0.253,...,0.3,1.1,1.4,2.2,0.6,0.0,1.1,1.2,6.5,2
12,A.J. Price,PG,25,IND,44,568,11.5,0.454,0.506,0.201,...,0.3,1.1,1.4,2.0,0.5,0.0,0.7,0.7,3.9,3
13,A.J. Price,PG,26,WAS,57,1278,12.4,0.501,0.484,0.150,...,0.4,1.6,2.0,3.6,0.6,0.1,1.1,1.3,7.7,4
30,Aaron Gray,C,23,CHI,61,613,13.9,0.529,0.005,0.485,...,1.1,1.6,2.8,0.7,0.3,0.3,0.9,1.9,4.3,1


In [90]:
fifthyearfull.head()

,Player,Pos,Age,Tm,G,MP_total,PER,TS%,3PAr,FTr,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Seasons_number
14,A.J. Price,SG,27,MIN,28,99,9.7,0.469,0.478,0.043,...,0.0,0.3,0.4,0.5,0.0,0.0,0.3,0.2,1.6,5
34,Aaron Gray,C,27,TOR,49,813,11.1,0.526,0.000,0.292,...,1.7,4.0,5.7,0.6,0.4,0.3,1.0,2.6,3.9,5
84,Adonal Foyle,C,26,GSW,79,1485,12.5,0.445,0.000,0.242,...,1.9,3.0,4.9,0.5,0.5,2.1,1.0,2.3,4.8,5
104,Adrian Griffin,SG,29,HOU,19,133,4.5,0.278,0.111,0.222,...,0.1,0.9,1.0,0.5,0.4,0.1,0.2,0.9,0.6,5
113,Al Harrington,SF,22,IND,82,2467,12.4,0.493,0.051,0.306,...,1.9,4.3,6.2,1.5,0.9,0.4,2.0,3.4,12.2,5


In [99]:
firstfouryearsfull.groupby('Player').mean()

,Age,G,MP_total,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Seasons_number
Player,,,,,,,,,,,,,,,,,,,,,
A.J. Price,24.5,51.75,876.50,12.150,0.48475,0.48700,0.20400,2.025,8.775,5.375,...,0.300,1.300,1.600,2.425,0.575,0.050,1.000,1.025,6.350,2.5
Aaron Gray,24.5,47.50,542.50,13.250,0.53850,0.00275,0.38050,14.025,22.325,18.125,...,1.350,2.150,3.550,0.650,0.300,0.300,0.750,2.075,3.550,2.5
Adonal Foyle,23.5,58.25,1095.25,11.550,0.44750,0.00150,0.30100,11.475,18.925,15.025,...,2.025,3.050,5.075,0.525,0.325,1.600,0.975,2.225,4.325,2.5
Adrian Griffin,26.5,62.00,1265.00,12.200,0.48575,0.15675,0.21525,6.975,15.350,11.075,...,1.200,2.500,3.675,1.575,1.075,0.150,0.750,2.150,5.100,2.5
Al Harrington,19.5,48.25,1054.75,10.425,0.47200,0.05925,0.31875,9.350,14.575,12.050,...,1.400,2.650,4.075,0.975,0.600,0.250,1.375,2.625,7.325,2.5
Al Horford,22.5,76.50,2582.75,17.950,0.57125,0.00350,0.29625,9.200,24.125,16.675,...,2.650,6.925,9.550,2.425,0.750,1.100,1.550,2.850,12.775,2.5
Al Jefferson,21.5,70.25,1837.75,18.825,0.54150,0.00575,0.33175,12.200,23.925,18.025,...,2.625,5.250,7.900,0.875,0.600,1.150,1.500,2.900,12.900,2.5
Al-Farouq Aminu,21.5,75.75,1760.00,11.775,0.50375,0.15450,0.29275,7.250,19.925,13.550,...,1.425,4.025,5.475,1.125,0.950,0.500,1.325,1.850,6.525,2.5
Alec Burks,21.5,57.00,1292.00,13.575,0.52100,0.19050,0.40300,4.050,10.825,7.425,...,0.775,2.250,3.000,2.000,0.625,0.175,1.475,2.000,10.525,2.5


In [100]:
firstfouryearsfull[firstfouryearsfull['Player']=='A.J. Price'].rolling(2, on='G').sum()['G']

TypeError: cannot handle this type -> object

In [118]:
meanfouryears = firstfouryearsfull.groupby('Player').mean().sort_index()

In [123]:
fifthyear_player_index = fifthyearfull.set_index('Player').sort_index()

In [124]:
meanfouryears

,Age,G,MP_total,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Seasons_number
Player,,,,,,,,,,,,,,,,,,,,,
A.J. Price,24.5,51.75,876.50,12.150,0.48475,0.48700,0.20400,2.025,8.775,5.375,...,0.300,1.300,1.600,2.425,0.575,0.050,1.000,1.025,6.350,2.5
Aaron Gray,24.5,47.50,542.50,13.250,0.53850,0.00275,0.38050,14.025,22.325,18.125,...,1.350,2.150,3.550,0.650,0.300,0.300,0.750,2.075,3.550,2.5
Adonal Foyle,23.5,58.25,1095.25,11.550,0.44750,0.00150,0.30100,11.475,18.925,15.025,...,2.025,3.050,5.075,0.525,0.325,1.600,0.975,2.225,4.325,2.5
Adrian Griffin,26.5,62.00,1265.00,12.200,0.48575,0.15675,0.21525,6.975,15.350,11.075,...,1.200,2.500,3.675,1.575,1.075,0.150,0.750,2.150,5.100,2.5
Al Harrington,19.5,48.25,1054.75,10.425,0.47200,0.05925,0.31875,9.350,14.575,12.050,...,1.400,2.650,4.075,0.975,0.600,0.250,1.375,2.625,7.325,2.5
Al Horford,22.5,76.50,2582.75,17.950,0.57125,0.00350,0.29625,9.200,24.125,16.675,...,2.650,6.925,9.550,2.425,0.750,1.100,1.550,2.850,12.775,2.5
Al Jefferson,21.5,70.25,1837.75,18.825,0.54150,0.00575,0.33175,12.200,23.925,18.025,...,2.625,5.250,7.900,0.875,0.600,1.150,1.500,2.900,12.900,2.5
Al-Farouq Aminu,21.5,75.75,1760.00,11.775,0.50375,0.15450,0.29275,7.250,19.925,13.550,...,1.425,4.025,5.475,1.125,0.950,0.500,1.325,1.850,6.525,2.5
Alec Burks,21.5,57.00,1292.00,13.575,0.52100,0.19050,0.40300,4.050,10.825,7.425,...,0.775,2.250,3.000,2.000,0.625,0.175,1.475,2.000,10.525,2.5


In [125]:
meanfouryears.index.equals(fifthyear.index)

True

In [128]:
rf = RandomForestRegressor(n_jobs = -1)

In [145]:
X = meanfouryears[statstokeep].values

In [134]:
y = fifthyear.pop('WS')

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .15, random_state=42)

In [157]:
rf = rf.fit(X_train, y_train)

In [158]:
y_predict = rf.predict(X_test)

In [159]:
y_predict

array([ 3.19,  3.94,  3.33,  0.55,  5.64,  2.91,  1.08,  2.52,  1.26,
        4.34,  2.43,  1.13,  7.77,  4.28,  1.31,  6.53,  2.1 ,  4.83,
       12.89,  2.74,  3.97,  7.47,  1.86,  2.63,  1.79, 11.36,  2.15,
        2.84,  3.59,  2.01,  6.89,  4.82,  1.66,  2.46,  4.85,  0.58,
        3.  ,  3.51,  4.06,  3.75,  3.82,  0.35,  2.16,  3.4 ,  5.08,
       13.06,  2.7 ,  0.3 ,  8.08,  6.11,  4.22,  3.33,  2.81,  0.5 ,
        9.44])

In [160]:
from sklearn.metrics import r2_score

Ouch, R2 score of .386 is not very good.

In [162]:
r2_score(y_test, y_predict)

0.3857110147986298

In [163]:
y_test - y_predict

Player
Gerald Henderson         0.41
Anthony Morrow          -3.24
Amir Johnson             1.17
Kevin Ollie              1.85
Brad Miller              3.96
Erick Strickland        -2.11
C.J. Miles               0.82
Dan Gadzuric            -1.72
Didier Ilunga-Mbenga    -0.96
Darius Miles            -3.04
Linton Johnson          -2.43
Aron Baynes              1.87
Kenyon Martin           -2.47
Boris Diaw              -1.28
DeSagana Diop            2.79
Kenneth Faried          -1.03
A.J. Price              -2.10
John Henson             -2.13
Marc Gasol              -1.39
Chuck Hayes              0.86
Alex Len                 0.53
Dwyane Wade             -4.17
Bruce Bowen              2.34
Jason Williams           2.07
Jason Kapono             0.11
Giannis Antetokounmpo    0.54
J.J. Redick              2.35
Juan Dixon              -1.14
Luther Head             -2.99
Marcus Morris            3.39
Andrew Bogut             0.61
Courtney Lee            -1.52
Austin Daye             -1.56
Mam

Let's see what the score is for every stat.

In [164]:
X = meanfouryears.values

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .15, random_state=42)

In [168]:
rf = rf.fit(X_train, y_train)

In [169]:
y_predict = rf.predict(X_test)

In [170]:
r2_score(y_test, y_predict)

0.4251985595375529

These scores really suck, so what happens if I predict WS/48, since that's "normalized" and can help account for injurys (see, e.g., Al Horford being far overpredicted, since his actual is really low because of injury--he only played 11 games his 5th year)

In [176]:
X = meanfouryears[statstokeep].values
y = fifthyear.pop('WSp48')

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .15, random_state=42)
rf = rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
r2_score(y_test, y_predict)

0.3060042715455883

In [178]:
y_test - y_predict

Player
Gerald Henderson        -0.0165
Anthony Morrow          -0.0005
Amir Johnson             0.0638
Kevin Ollie             -0.0234
Brad Miller              0.0394
Erick Strickland        -0.0312
C.J. Miles               0.0087
Dan Gadzuric            -0.0966
Didier Ilunga-Mbenga    -0.0244
Darius Miles            -0.0845
Linton Johnson          -0.0586
Aron Baynes              0.0843
Kenyon Martin           -0.0140
Boris Diaw              -0.0421
DeSagana Diop            0.0019
Kenneth Faried          -0.0064
A.J. Price              -0.0675
John Henson              0.0101
Marc Gasol              -0.0315
Chuck Hayes              0.0601
Alex Len                 0.0211
Dwyane Wade             -0.0548
Bruce Bowen              0.0076
Jason Williams           0.0215
Jason Kapono             0.0054
Giannis Antetokounmpo    0.0262
J.J. Redick              0.0551
Juan Dixon              -0.0220
Luther Head             -0.0441
Marcus Morris            0.0070
Andrew Bogut             0.0296
C

In [184]:
last_train_season = 4
nums = set(range(1,last_train_season+1))

In [185]:
nums

{1, 2, 3, 4}

In [195]:
import importlib
importlib.reload(seasonalregressor)

<module 'source.seasonalregressor' from '/Users/kv/workspace/kv-capstone/NBAwinshare/source/seasonalregressor.py'>

In [196]:
sr = seasonalregressor.SeasonalRegressor()

In [197]:
X, y, players = sr.create_train_and_predict_X_and_y_for_season_range(fullstats, demographic)

Number of players:  578  with full season data for seasons: {1, 2, 3, 4, 5}
Indices of train set and to-predict set MATCH
Using all columns


In [200]:
X

array([[2.45000e+01, 5.17500e+01, 8.76500e+02, ..., 1.02500e+00,
        6.35000e+00, 2.50000e+00],
       [2.45000e+01, 4.75000e+01, 5.42500e+02, ..., 2.07500e+00,
        3.55000e+00, 2.50000e+00],
       [2.35000e+01, 5.82500e+01, 1.09525e+03, ..., 2.22500e+00,
        4.32500e+00, 2.50000e+00],
       ...,
       [2.05000e+01, 7.07500e+01, 1.63475e+03, ..., 2.77500e+00,
        8.35000e+00, 2.50000e+00],
       [3.05000e+01, 4.65000e+01, 7.17000e+02, ..., 2.40000e+00,
        5.77500e+00, 2.50000e+00],
       [2.65000e+01, 2.65000e+01, 3.38000e+02, ..., 1.37500e+00,
        3.67500e+00, 2.50000e+00]])